A neural network consist of cnn layer (Yoon Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [3]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [5]:
len(labels)

78

In [6]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [7]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [8]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [9]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [10]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [11]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [12]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [13]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [14]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [15]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [ ]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    # embd_sentence = Permute((2,1))(embd_sentence)
    # embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # lstm
    lstm_1 = LSTM(128, return_sequences=True,name='lstm_1')(embd_sentence)
    # drop_out_1 = Dropout(0.5,name='drop_out_1')(lstm_1)
    lstm_2 = LSTM(64,name='lstm_2')(lstm_1)
    drop_out_2 = Dropout(0.4,name='drop_out_2')(lstm_2)

    # dense
    dense_1 = Dense(256,name='dense_1')(drop_out_2)
    dense_2 = Dense(128,name='dense_2')(dense_1)
    dense_3 = Dense(len(labels),activation='sigmoid',name='dense_3')(dense_1)
    out = Softmax(name='out')(dense_3) 
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=["accuracy"]) 
    return model


model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())

Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 300, 128)          219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
drop_out_2 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_3 (Dense)              (None, 78)    

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=50,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/50
405/405 [==============================] - 308s 752ms/step - loss: 0.1133 - accuracy: 0.0169 - val_loss: 0.1169 - val_accuracy: 0.0213

Epoch 00001: val_loss improved from inf to 0.11694, saving model to best_cnn_4fc.h5
Epoch 2/50
405/405 [==============================] - 306s 756ms/step - loss: 0.1118 - accuracy: 0.0244 - val_loss: 0.1169 - val_accuracy: 0.0223

Epoch 00002: val_loss improved from 0.11694 to 0.11692, saving model to best_cnn_4fc.h5
Epoch 3/50
405/405 [==============================] - 310s 766ms/step - loss: 0.1120 - accuracy: 0.0252 - val_loss: 0.1169 - val_accuracy: 0.0264

Epoch 00003: val_loss improved from 0.11692 to 0.11690, saving model to best_cnn_4fc.h5
Epoch 4/50
405/405 [==============================] - 310s 765ms/step - loss: 0.1145 - accuracy: 0.0217 - val_loss: 0.1169 - val_accuracy: 0.0278

Epoch 00004: val_loss improved from 0.11690 to 0.11688, saving model to best_cnn_4fc.h5
Epoch 5/50
405/405 [==============================] - 312s 771ms

# ***If the model is generated before:***

In [16]:
model = load_model('Lstm_3_just_accuracy.h5')
# model.save('Lstm_3_just_accuracy.h5')

In [17]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [18]:
print(pred[0])#example

[0.01452335 0.01598062 0.01668891 0.00816385 0.00879594 0.00781629
 0.01716331 0.00862093 0.00828637 0.01350479 0.01709841 0.00971079
 0.01524494 0.00811072 0.00793961 0.01610098 0.0123339  0.01652976
 0.0132545  0.0097207  0.01374433 0.01548942 0.00877267 0.01651628
 0.01084339 0.01815191 0.01612981 0.01727338 0.01447666 0.01596189
 0.00964683 0.01132933 0.01538491 0.01666639 0.01175464 0.01710004
 0.01527705 0.0101454  0.00815263 0.00834053 0.0166777  0.01195076
 0.01005019 0.01644499 0.01784715 0.00797466 0.00817356 0.01583368
 0.01560309 0.00985477 0.00808483 0.0082297  0.0089913  0.01151126
 0.01495847 0.00834521 0.00902286 0.00930542 0.01595257 0.01638179
 0.01652593 0.00874774 0.01590862 0.01652034 0.01062724 0.01764767
 0.00889372 0.01239979 0.01647253 0.0163764  0.01558    0.01693973
 0.01452838 0.00845623 0.00806998 0.01599721 0.00897282 0.01339554]


In [25]:
y_pred=[]
for l in pred:
  temp=[]
  for value in l:
    if value>=np.mean(pred[0]) + .5*np.sqrt(np.var(pred[0])):
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [27]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.0
              precision    recall  f1-score   support

           0       0.03      0.04      0.03       115
           1       0.04      1.00      0.08       177
           2       0.04      0.98      0.08       173
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00        75
           5       0.00      0.00      0.00        13
           6       0.05      0.99      0.09       189
           7       0.00      0.00      0.00        38
           8       0.00      0.00      0.00        27
           9       0.00      0.00      0.00       124
          10       0.03      0.99      0.06       139
          11       0.00      0.00      0.00        66
          12       0.03      1.00      0.07       138
          13       0.00      0.00      0.00        20
          14       0.00      0.00      0.00        18
          15       0.04      0.96      0.07       156
          16       0.00      0.00      0.00       121
          17 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
